In [1]:
import csv
from mysql.connector import MySQLConnection, Error
from configparser import ConfigParser
from pandas import DataFrame
import pandas as pd

In [2]:
def read_config(config_file = 'config.ini', section = 'mysql'):
    parser = ConfigParser()
    parser.read(config_file)
    
    config = {}
    
    if parser.has_section(section):
        # Parse the configuration file.
        items = parser.items(section)
        
        # Construct the parameter dictionary.
        for item in items:
            config[item[0]] = item[1]
            
    else:
        raise Exception(f'Section [{section}] missing ' + \
                        f'in config file {filename}')
    
    return config

In [3]:
db_config = read_config() # Calling read config function

In [4]:
def make_connection(config_file = 'config.ini', section = 'mysql'): #function to create connection
    try:
        db_config = read_config(config_file, section)            
        conn = MySQLConnection(**db_config)
        
        if conn.is_connected():
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [5]:
conn = make_connection() # calling connection function
conn

In [6]:
cursor = conn.cursor()

In [7]:
cursor.execute('DROP TABLE IF EXISTS STUDENT')
cursor.execute('DROP TABLE IF EXISTS STUDENTCOURSES')
cursor.execute('DROP TABLE IF EXISTS COURSES')
cursor.execute('DROP TABLE IF EXISTS TEACHER')
cursor.execute('DROP SCHEMA IF EXISTS CLASS')

In [8]:
cursor.execute('CREATE SCHEMA CLASS')

In [9]:
def do_query_return_all(conn, sql): # function to execte the query
    cursor = None
    
    try:
        # Do the query.
        cursor = conn.cursor()
        cursor.execute(sql);

        # Return the fetched data as a list of tuples,
        # one tuple per table row.
        rows = cursor.fetchall()
        count = cursor.rowcount

        cursor.close()
        return [rows, count]

    except Error as e:
        print('Query failed')
        print(e)

        cursor.close()
        return [(), 0]

### Generate SQL CREATE TABLE commands for your tables. Add foreign key constraints. Use the commands in a Python program or notebook. 

#### There are 4 tables:
<ul>
    <b><li>Student</li>
    <li>StudentCourses</li>
    <li>Teacher</li>
        <li>Courses</li></b>
</ul>

In [10]:
sql1 = ( "CREATE TABLE CLASS.Student \n"
      + '(\n'
      +   'StudentID INT NOT NULL, \n'
      +    'FirstName VARCHAR(40) NOT NULL, \n'
      +    'LastName VARCHAR(40) NOT NULL, \n'
      +    'Gender CHAR(10) NOT NULL, \n'
      +    'PRIMARY KEY (StudentID) \n'
      + ')'
      )
print(sql1)
rows, count = do_query_return_all(conn, sql1)

CREATE TABLE CLASS.Student 
(
StudentID INT NOT NULL, 
FirstName VARCHAR(40) NOT NULL, 
LastName VARCHAR(40) NOT NULL, 
Gender CHAR(10) NOT NULL, 
PRIMARY KEY (StudentID) 
)


 ### using named constraints, create named constraints for Student table

In [11]:
sql2 = ( "CREATE TABLE CLASS.StudentCourses \n"
      + '(\n'
      +  'StudentID VARCHAR(5) NOT NULL,\n'
      + 'CourseTitle VARCHAR(20) NOT NULL,\n'
      + 'Grade VARCHAR(20) NOT NULL,\n'
      + 'CONSTRAINT chk PRIMARY KEY(StudentID, CourseTitle)\n'  
      + ')'
       )
print(sql2)
rows, count = do_query_return_all(conn, sql2)

CREATE TABLE CLASS.StudentCourses 
(
StudentID VARCHAR(5) NOT NULL,
CourseTitle VARCHAR(20) NOT NULL,
Grade VARCHAR(20) NOT NULL,
CONSTRAINT chk PRIMARY KEY(StudentID, CourseTitle)
)


In [12]:
sql4 = ( "CREATE TABLE CLASS.Teacher \n"
      + '(\n'
  + 'TeacherID VARCHAR(20) NOT NULL,\n'
  + 'TeacherName VARCHAR(40) NOT NULL,\n'
  + 'PRIMARY KEY (TeacherID) \n'
      + ')'
       )
print(sql4)
rows, count = do_query_return_all(conn, sql4)

CREATE TABLE CLASS.Teacher 
(
TeacherID VARCHAR(20) NOT NULL,
TeacherName VARCHAR(40) NOT NULL,
PRIMARY KEY (TeacherID) 
)


In [13]:
sql3 = ( "CREATE TABLE CLASS.Courses \n"
      + '(\n'
      + 'CourseTitle VARCHAR(20) NOT NULL,\n'
      + 'Fee VARCHAR(10) NOT NULL,\n'
      + 'Qualification VARCHAR(30) NOT NULL,\n'
      + 'TeacherID VARCHAR(20) NOT NULL, \n'
      + 'PRIMARY KEY (CourseTitle) \n'
    #  + 'FOREIGN KEY (TeacherID) REFERENCES Teacher(TeacherID)\n'
    #  + 'FOREIGN KEY (CourseTitle) REFERENCES Courses(CourseTitle)\n'
      + ')'
       )
print(sql3)
rows, count = do_query_return_all(conn, sql3)

CREATE TABLE CLASS.Courses 
(
CourseTitle VARCHAR(20) NOT NULL,
Fee VARCHAR(10) NOT NULL,
Qualification VARCHAR(30) NOT NULL,
TeacherID VARCHAR(20) NOT NULL, 
PRIMARY KEY (CourseTitle) 
)
